# Timber Properties for use in design

In [1]:
import pandas as pd
timberMaterials = pd.read_csv('timberProperties.csv', index_col=0)
display(timberMaterials)

,fmk,fvk,ft0k,ft90k,fc0k,fc90k,E0m,E005,Gm,pm,pk
C16,16,1.8,10,0.5,17,2.2,8000,5400,500,370,310
C24,24,2.5,14,0.5,21,2.5,11000,7400,690,420,350


# Capacities of sections

## Bending

The bending capacity of a timber section is given by:

$$ M_{Rd} = \frac{k_{mod} k_h k_{sys} k_{crit} f_{mk}}{\gamma_m} Z $$

In [2]:
import geometricProperties

ModuleNotFoundError: No module named 'geometricProperties'

In [6]:
# section bending moment capacity
fmk = timberMaterials.loc['C24','fmk']
kmod = 0.60 # for permanent loading
kcrit = 1 # check for long unrestrained beams
b = 50
d = 195
kh = min(1.3, (150/d)**0.20) # for solid timber only - check for glulam etc
ksys = 1 # can take 1.1 for load sharing systems
gm = 1.30 # for solid softwood timber
Z = b*d**2/6
MRd = kcrit * ksys * kh * kmod * fmk * Z / gm * 10**-6
print('For ksys = {:.2f}, kh = {:.2f}, kmod = {:.2f}, fmk = {:.2f}, b = {:.2f}, d = {:.2f}, gm = {:.2f}\nMRd = {:.2f} kNm'.format(ksys, kh, kmod,fmk,b,d,gm,MRd))

For ksys = 1.00, kh = 0.95, kmod = 0.60, fmk = 24.00, b = 50.00, d = 195.00, gm = 1.30
MRd = 3.33 kNm


## Shear

The shear capacity of a timber section is given by:

$$ V_{Rd} = \frac{k_{mod} f_{vk}}{\gamma_m} \frac{2}{3} bd $$

In [7]:
# section shear capacity
fvk = timberMaterials.loc['C24','fvk']
b = 50
d = 195
kmod = 0.60
gm = 1.30
VRd = kmod* fvk * 2 * b * d / (3 * gm) * 10**-3
print('For fvk = {:.2f}, kmod = {:.2f}, gm = {:.2f}, b = {:.2f}, d = {:.2f}\nVRd = {:.2f}kN'.
     format(fvk, kmod, gm, b, d, VRd))



For fvk = 2.50, kmod = 0.60, gm = 1.30, b = 50.00, d = 195.00
VRd = 7.50kN


## Axial Compression

\begin{align}
\lambda_{rel,y} = \frac{\lambda_y}{\pi} \sqrt{\frac{f_{c,0,k}}{E_{0.005}}} \text{ Where } \lambda_y = \frac{le_y}{r_y}\\
\lambda_{rel,z} = \frac{\lambda_z}{\pi} \sqrt{\frac{f_{c,0,k}}{E_{0.005}}} \text{ Where } \lambda_z = \frac{le_z}{r_z}\\
\end{align}

# Racking Shear Calculations

The design racking strength of a wall diaphragm $F_{i,v,Rd} = K_{opening} K_{i,w} f_{p,d,t} L$

Where: $f_{p,d,t} = f_{p,d,1} + K_{comb} f_{p,d,2}$

The design shear capacity per unit length of the perimeter fasteners to the sheathing ($f_{p,d}$) is given by:

$$f_{p,d} = \frac{1.2 F_{t,Rd} (1+2.5s)}{s}$$

Where $F_{f,Rd}$ is the design capacity of a single fixing.

# Connections

## Withdrawal Capacities

In [2]:
withdrawalData = {'C16':[5.09,5.38,5.77,5.96,6.44,7.21,7.69],
                 'C24':[6.49,6.86,7.35,7.60,8.21,9.19,9.80]}
withdrawal = pd.DataFrame(withdrawalData)
nailDiameters = [2.65, 2.80, 3, 3.1, 3.35, 3.75, 4]
withdrawal.index = nailDiameters
print('Characteristic withdrawal resistance for smooth nails:')
display(withdrawal)

Characteristic withdrawal resistance for smooth nails:


,C16,C24
2.65,5.09,6.49
2.80,5.38,6.86
3.00,5.77,7.35
3.10,5.96,7.60
3.35,6.44,8.21
3.75,7.21,9.19
4.00,7.69,9.80


The withdrawl load of a screw is given by:

$$R_{d,withdrawal} = \frac{4t_{pen}nk_{mod}R_k}{\gamma_M}$$

In [5]:
# axial withdrawal load for a screw 
# tpen >= 8 x nominal diameter
grade = 'C24'
n = 1
d = 4
Rk = withdrawal.loc[d,grade]
kmod = 0.6
tpen = 120
gm = 1.3

Rdw = 4 * tpen * n * kmod * Rk / gm
print('Withdrawl capacity of a {:.2f}mm diameter nail with a pointside penetration of {:.2f}mm is {:.2f}N'.
      format(d, tpen, Rdw))


Withdrawl capacity of a 4.00mm diameter nail with a pointside penetration of 120.00mm is 2171.08N


# Force effects

## Deflection

Combined bending and shear deflection is a simply supported beam is given by:

$$\delta = \frac{5wl^4}{384EI} + \frac{V_{app}}{8 G_m bd}$$

In [47]:
grade_ref = 'C24'
E = timber_properties.loc[grade_ref, 'E0m']